In [ ]:
# !pip install -U plotly
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

In [1]:
import gc
from collections import Counter
import plotly.io as pio
pio.renderers.default = "pdf"
import pandas as pd
pd.options.plotting.backend = "plotly"
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

In [2]:
products = pd.read_csv('/content/drive/My Drive/instacart/products.csv')
aisle = pd.read_csv('/content/drive/My Drive/instacart/aisles.csv')
train_orders = pd.read_csv(
    '/content/drive/My Drive/instacart/order_products__train.csv'
    )
prior_orders = pd.read_csv(
    '/content/drive/My Drive/instacart/order_products__prior.csv'
    )
orders = pd.read_csv('/content/drive/My Drive/instacart/orders.csv')
departments = pd.read_csv('/content/drive/My Drive/instacart/departments.csv')

In [3]:
# Utility Functions 

def plot_histogram(df, x_column_name, y_column_name, title):
    if y_column_name is None:
        fig = px.histogram(df, x=x_column_name, title=title)
    else:
        fig = px.histogram(df, x=x_column_name, y=y_column_name,
                           title=title, color=y_column_name)
    fig.update_layout(
        bargap=0.2
    )
    fig.update_xaxes(tickangle=45)
    fig.show()

def downcast_df(df):
    """
    As in these dataframes, all the float values dont contain any decimal
    thus converting them to int inorder to save RAM space
    """
    for column in df:
        if df[column].dtype != np.object:
            df[column] = df[column].astype(np.int8)
    return df

def product_aisle_top_k(aisle_id, top=10):
    fig = product_orders[product_orders['aisle_id'] == aisle_id]['product_name'].value_counts()[:top].plot(
        kind='bar',
        title='Top {} products sold on Aisle {} ({})'.format(
            top, aisle_id,
            aisle[aisle['aisle_id'] == aisle_id]['aisle'].tolist()[0]))
    fig.update_layout(
        xaxis_title = 'Products',
        yaxis_title = 'Sell Count'
    )
    fig.update_xaxes(tickangle=45)
    fig.show()


def get_max_ordinal_number(product):
    temp_df = product_orders[(product_orders['product_name'] == product)& (product_orders['reordered'] == 1)]
    cart_ordinal = temp_df['add_to_cart_order'].values.tolist()
    cart_ordinal = dict(Counter(cart_ordinal))
    cart_ordinal = dict(
        sorted(cart_ordinal.items(), key=lambda x: x[1], reverse=False))
    max_ordinal = cart_ordinal.popitem()[0]
    del cart_ordinal
    del temp_df
    gc.collect()
    return max_ordinal

In [4]:
filtered_orders_prior = orders[orders['eval_set'] == 'prior']
filtered_orders_prior.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [5]:
plot_histogram(filtered_orders_prior, 'order_dow', None,
               title='Histogram of Day of week when people  prefer to shop')

<b>Observations</b>
- On seeing the plots, I observe that people tend to shop more on days 0 and 1.
- We dont have mapping to what days these number represent, but seeing the graph, I assume that 0 and 1 maybe refer to weekends as people are more free on weekends which can lead them to go shopping.

In [6]:
plot_histogram(filtered_orders_prior, 'order_hour_of_day', None,
               title='Histogram of what day hour do people  prefer to shop')

<b>Observations</b>
- From this plot I conclude that people tend to shop more between 10am - 4pm i.e in day time, compared to night time or midnight.

In [7]:
grouped_orders = orders.groupby(
    ["order_dow", "order_hour_of_day"]
    ).agg({'order_number': 'count'})
grouped_orders.reset_index(inplace=True)
grouped_orders = grouped_orders.pivot('order_dow', 'order_hour_of_day', 'order_number')
fig = px.imshow(grouped_orders, title='Pivot Plot between Days of weeks VS Time of Day')
del grouped_orders
gc.collect()
fig.show()


- In previous orders_dow plot we established that 0 and 1 are weekends, lets say 0 is saturday and 1 is sunday
- Here I observed that on Saturday(0), people tend to shop more between 12pm - 3pm
- While one Sunday(1) people tend to shop early in the morning betwen 9am - 11am

In [8]:
days_order = filtered_orders_prior[~filtered_orders_prior['days_since_prior_order'].isna() ==True]
plot_histogram(days_order, 'days_since_prior_order', None,
               'After how many days user tend to come back for shopping')
del days_order
gc.collect()

733

<b>Observation</b>
- People tend to shop again after 7 days or 30 days.
- 7 days is obvious as from the graph of 'days of weeks' we saw that people tend to shop more on weeknds.
- But its not necessary that its always weekend, people may buy sometime during weekdays and repeat that pattern every 7 days.
- Also we can observe that people often tend to come between 2-7 days again, by this I am assuming that customers may be buying household groceries often like milks, fruits, eggs, breads etc which can be utilised within a week.

In [9]:
# Merge the Prior orders with the products data to get respective Aisle and 
# Department of the product purchased by the user
product_orders = prior_orders.merge(products, on='product_id', how='left')
product_orders = product_orders.merge(departments, how='left', on='department_id')
product_orders = product_orders.merge(aisle, how='left', on='aisle_id')
product_orders.drop(['department_id'], inplace=True, axis=1)


# Find the aisle(s) which has most number of product reorders
aisle_groupby = product_orders.groupby('aisle_id').agg({'reordered':'sum'})
aisle_groupby.reset_index(inplace=True) 

# Plot the bar char with where number of orders per aisle is used as
# color palatte 
px.bar(aisle_groupby, 'aisle_id', 'reordered', 'reordered',
       title='Histograms of Aisles used by people according to the reodered product')


In [10]:
del aisle_groupby
gc.collect()

26

**Observations**
- People tend to reorder from Aisle 24, 83, 120, and 124 the most when they shop
- Aisle 24 is related to fresh fruits
- Aisle 83 is related to fresh vegetables
- Aisle 120 is for Yogurt where different flavours of Yogurt are selled.
- Aisle 123 consists of Packed Vegetables and fruits
- This is obvious as people tend to buy fresh fruits, vegetables, yogurts on regular basis instead of stocking them up in their kitchen



Based on the observations made above, lets check those 4 aisles and find which products among those aisles are purchased more by the users

In [11]:
product_aisle_top_k(24)

In [12]:
product_aisle_top_k(83)

In [13]:
product_aisle_top_k(120)

In [14]:
product_aisle_top_k(123)

<b> Observations </b>
- From all these plots we can observe that the mose purchased product is Banana, Organic Banana, strawberies, yogurts, and other packed organic vegetables
- These plots display the most ordered products irrelevant of reorders.

In [15]:
# Get the deparments from which we have most of the reorders
department_group = product_orders.groupby('department').agg(
    {'reordered': 'sum'}
    )
department_group.reset_index(inplace=True)
plot_histogram(department_group, 'department', 'reordered',
               'Departments which has most number of product reorders')

In [16]:
top_20_reorderd = product_orders[product_orders['reordered'] == 1]['product_name'].value_counts()[:20].keys().tolist()
product_orders[product_orders['product_name'].isin(top_20_reorderd)]['department'].value_counts()

produce       3110110
dairy eggs     214265
Name: department, dtype: int64

**Observations**
- By this graph and above output we can confirm that most of the reordered products come from produce and dairy eggs department

<b>Now get the products which are most reordered </b>

In [17]:
# product_orders DF contains of product info along with the aisles and department.

asd = product_orders[product_orders['reordered'] == 1].groupby(
    'product_name').agg({'product_name':'count'})
asd.columns = ['count']
asd.reset_index(inplace=True)
asd.sort_values('count', inplace=True, ascending=False)
plot_histogram(asd[:10], 'product_name', 'count', title="Top 10 Reoredered Items")


In [18]:
del asd
gc.collect()

2186

In [19]:
# Check the basket size when the customer comes to checkout
basket_size = product_orders.groupby('order_id').agg({'add_to_cart_order': 'count'})
basket_size.columns = ['basket_size']
basket_size.reset_index(inplace=True)
plot_histogram(basket_size, 'basket_size', None,
               'Number of products in the basket when user checks out')


In [20]:
del basket_size
gc.collect()

733

**Observations**
- User mostly checkout with 4-7 products in their cart, mostly its 5

\
Now from top 20 reorderd products, I will get the ordrinal of when the user puts that product in the cart.
\
Meaning I will take the add_to_cart_order of the most reorderd product and observe when the user puts that particular product in his / her basket.

In [21]:
temp = pd.DataFrame()
count = []
for p in top_20_reorderd:
    count.append(get_max_ordinal_number(p))

temp['product'] = top_20_reorderd
temp['max_basket_ordinal'] = count
temp

,product,max_basket_ordinal
0,Banana,1
1,Bag of Organic Bananas,1
2,Organic Strawberries,2
3,Organic Baby Spinach,2
4,Organic Hass Avocado,2
5,Organic Avocado,2
6,Organic Whole Milk,1
7,Large Lemon,3
8,Organic Raspberries,2
9,Strawberries,2


**Observations**
- By observing the upper Dataframe I conclude that most of these product are the first or second choosen product added to cart by user only some products are 3rd priority to user while adding them to the cart.

\
Seeing this, lets plot the graph of the add_to_cart_order, and see if it is the actual case that the items added 1st or 2nd are most reordered or not

In [22]:
reorder_cart = product_orders.groupby('add_to_cart_order').agg(
    {'reordered':'sum'}
    )
reorder_cart.columns = ['count']
reorder_cart = reorder_cart.reset_index()
fig = go.Figure(data=go.Scatter(
    x=reorder_cart['add_to_cart_order'],
    y=reorder_cart['count'],
    mode='lines+markers'))
fig.show()


My earlier prediction seems correct, products which are added first are more reorderd, which is true as per human nature we as people tend to first put the product which we really need.We saw that Banana, other fruits, yogurt are the things which people use in day-to-day life which is why they are added first in the cart and reordered many a time as well.

In [23]:
orders_grp = prior_orders.groupby("order_id").agg({'reordered': 'sum'})
orders_grp.reset_index(inplace=True)
orders_grp['reordered'].value_counts().get(0) / orders_grp.shape[0]

0.12084859313304347

In [24]:
orders_grp = train_orders.groupby("order_id").agg({'reordered': 'sum'})
orders_grp.reset_index(inplace=True)
orders_grp['reordered'].value_counts().get(0) / orders_grp.shape[0]

0.06555952716658156

- From these outputs we can say that in prior_orders there are 12.08% products and in train there are 6.5% products which are not reorderd

In [25]:
temp = pd.merge(filtered_orders_prior[['order_id', 'order_dow', 'order_hour_of_day']],
                product_orders[['product_id', 'order_id', 'product_name', 'reordered']],
                how='left', on='order_id')

In [26]:
temp.head()

,order_id,order_dow,order_hour_of_day,product_id,product_name,reordered
0,2539329,2,8,196,Soda,0
1,2539329,2,8,14084,Organic Unsweetened Vanilla Almond Milk,0
2,2539329,2,8,12427,Original Beef Jerky,0
3,2539329,2,8,26088,Aged White Cheddar Popcorn,0
4,2539329,2,8,26405,XL Pick-A-Size Paper Towel Rolls,0


In [27]:
# Get the reordered sum for each order_hour_of_day and its product
# thus we get the count of each product ordered on each day.
asd = temp.groupby(["order_hour_of_day", "product_name"]).agg({'reordered':'sum'})
asd.columns = ['count']
asd.reset_index(inplace=True)
asd.sort_values("count", inplace=True)

In [28]:
asd.groupby(['order_hour_of_day']).agg({'count':np.max})

,count
order_hour_of_day,
0,2315
1,1136
2,681
3,477
4,563
5,1108
6,4125
7,12607
8,23764


In [29]:
idx = asd.groupby(['order_hour_of_day'])['count'].transform(max) == asd['count']
asd[idx]

,order_hour_of_day,product_name,count
55882,3,Banana,477
68873,4,Banana,563
41385,2,Banana,681
81899,5,Banana,1108
23424,1,Bag of Organic Bananas,1136
23510,1,Banana,1136
1511,0,Banana,2315
98037,6,Banana,4125
742952,23,Banana,4486
713346,22,Banana,7539


In [30]:
px.bar(asd[idx], 'order_hour_of_day', 'count', hover_name='product_name', text='product_name',
       title='Count of Item sold most on the particular hour of the day for prior orders')

**Observations**
- In the prior data the highest number of sale on each hour of day is Banana
- On hour 1, Organic Banana and Banana are sold mostly in same Quantity

In [31]:
del asd
gc.collect()

668

In [32]:
# Get the reordered sum for each order_dow and its product
# thus we get the count of each product ordered on each day.
asd = temp.groupby(["order_dow", "product_name"]).agg({'reordered':'sum'})
asd.columns = ['count']
asd.reset_index(inplace=True)
asd.sort_values("count", inplace=True)

In [33]:
# Get the max count of a product ordered in a day
asd.groupby(['order_dow']).agg({'count':np.max})

,count
order_dow,
0,81347
1,74425
2,50188
3,44925
4,44479
5,50201
6,53044


In [34]:
idx = asd.groupby(['order_dow'])['count'].transform(max) == asd['count']
asd[idx]

,order_dow,product_name,count
185312,4,Banana,44479
140363,3,Banana,44925
95254,2,Banana,50188
230208,5,Banana,50201
275665,6,Banana,53044
49485,1,Banana,74425
3381,0,Banana,81347


**Observations**
- By this I conclude that on all days Banana is the most reordered product.
- This was obvious as we had banana to be highest sold product on each hour as seen earlier so it has to be Banana to be highest sold product in each day.
- Also on days 0 & 1 we have highest sale of Bananas.

In [35]:
px.bar(asd[idx], 'order_dow', 'count', hover_name='product_name', text='product_name',
       title='Count of Product sold highest on the particular day')